In [1]:
import csv
from dateparser import parse

with open('input.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    data = [x for x in list(csv_reader) if x['Szenarium'] != '']

In [2]:
def fix_location(x):
    if len(x['Bundesland']) != 0:
             return x
    for deli in ('/', ','):
        split_1 = x['Ort'].rsplit(deli)
        # print(split_1)
        if len(split_1) > 1:
                x['Bundesland'] = harm_land(split_1[-1].strip())
                x['Ort'] = split_1[0].strip()
                return x
    if x['Ort'] in ('Berlin', 'Hamburg', 'Bremen'):
        x['Bundesland'] = x['Ort']
    return x

In [3]:
abr_text = """BW 	Baden-Württemberg
BY 	Bayern
BE 	Berlin
BB 	Brandenburg
HB 	Bremen
HH 	Hamburg
HE 	Hessen
MV 	Mecklenburg-Vorpommern
NI 	Niedersachsen
NW 	Nordrhein-Westfalen
RP 	Rheinland-Pfalz
SL 	Saarland
SN 	Sachsen
ST 	Sachsen-Anhalt
SH 	Schleswig-Holstein
TH 	Thüringen"""
abr = [x.split() for x in abr_text.split("\n")]
lands = [x.split()[-1] for x in abr_text.split("\n")]
lands

['Baden-Württemberg',
 'Bayern',
 'Berlin',
 'Brandenburg',
 'Bremen',
 'Hamburg',
 'Hessen',
 'Mecklenburg-Vorpommern',
 'Niedersachsen',
 'Nordrhein-Westfalen',
 'Rheinland-Pfalz',
 'Saarland',
 'Sachsen',
 'Sachsen-Anhalt',
 'Schleswig-Holstein',
 'Thüringen']

In [4]:
bw=["Ba.-Württ.", "Baden-Württemb.", "Bad.-Württ.", "Bad.-Würt.", "Baden-Württ.", "Baden-Wbg."]
rp=["Rheinl.-Pfalz", "Rheinl.-Pf.", "Rhld.-Pfalz", "Rheinl.-Pfalz", "Rheinland-Pf."]
st=["Sachsen-Anh.", "Sn.-Anhalt"]
nr=["NRW", "Nordrhein-Westf."]
ni=["Nds.", "Nieders."]
mv=["MV", "M.V."]
sh=["Schleswig-Holst.", "Schl.-Hol.", "Schlesw.-Hol.", "Schleswig-H."]
bb=["Brbg", "Brbg."]
be=["Bln."]
he=["Hess."]
sa=["Sachs."]

In [5]:
def harm_land(x):
    if x in bw:
        return "Baden-Württemberg"
    if x in rp:
        return "Rheinland-Pfalz"
    if x in st:
        return "Sachsen-Anhalt"
    if x in nr:
        return "Nordrhein-Westfalen"
    if x in ni:
        return "Niedersachsen"
    if x in mv:
        return "Mecklenburg-Vorpommern"
    if x in sh:
        return "Schleswig-Holstein"
    if x in bb:
        return "Brandenburg"
    if x in be:
        return "Berlin"
    if x in he:
        return "Hessen"
    if x in sa:
        return "Sachsen"
    return x

In [6]:
d1 = [fix_location(x) for x in data]

In [7]:
for x in d1:
    if x['Bundesland'] not in lands:
        print(x['Bundesland'])

In [8]:
for x in d1:
    x['Datum'] = parse(x['Datum'], languages=['de']).date().isoformat()
    name = x['Name'] 
    name = name[0].upper() + name[1:]
    x['Name'] = name.replace('Unbek.', 'Unbekannter').replace('Unbek ', 'Unbekannter ').replace('Unbekannter Frau', 'Unbekannte Frau')

In [9]:
for x in d1:
    print(x['Name'])

Dennis J.
Unbekannter Mann
Joachim H.
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter US-Soldat
Unbekannter Mann
Unbekannter Mann
Tennessee Eisenberg
Sven B.
Dirk P.
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Jens F.
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Rumäne
Unbekannter Mann
Sven G.
Unbekannter Mann
Unbekannter Mann
Sebastian H.
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Wladimir S.
Unbekannter Mann
Unbekannte Frau
Jaroslav B.
Sabine R.
Engelbert L.
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
René B.
Unbekannter Mann
Unbekannter SEK-Beamter
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Unbekannter Mann
Marame Wegener
Polizeibeamter
Unbekannter Mann
Mike L.
Unbekannt

In [10]:
d2 = sorted(d1, key=lambda x: x['Datum'])

In [11]:
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=d2[0].keys())

    writer.writeheader()
    writer.writerows(d2)

In [12]:
d2[150:]

[{'Fall': '9',
  'Name': 'Unbekannter Mann',
  'Alter': '40',
  'Datum': '2017-05-22',
  'Ort': 'Bonn',
  'Bundesland': 'Nordrhein-Westfalen',
  'Szenarium': 'Die Polizei wird von Nach\xad\xadbarInnen gerufen, die in einer Wohnung lau\xadte Ge\xadräusche hören. Da niemand öffnet, be\xadtreten die Polizisten die Woh\xadnung über den Bal\xadkon. Drinnen finden sie die Leichen einer Frau und ihres Kindes. In einem anderen Zimmer hat sich der Vater ein\xadge\xadschlossen. Als die Beam\xadten die Tür gewalt\xadsam öffnen, kommt \xades zu „einer kurzen konfron\xadtati\xadven Situation“ in deren Verlauf der Mann er\xadschos\xadsen wird.',
  'Opfer mit Schusswaffe': 'Nein (weiteres unklar)',
  'Schusswechsel': 'Nein',
  'Sondereinsatzbeamte': 'Nein',
  'Verletzte/getötete Beamte': 'Nein',
  'Vorbereitete Polizeiaktion': 'Nein',
  'Anmerkung': '',
  'Interne Anmerkung': ''},
 {'Fall': '10',
  'Name': 'Mostafa J.',
  'Alter': '41',
  'Datum': '2017-06-03',
  'Ort': 'Arnschwang',
  'Bundesland': 